In [2]:
import os

In [3]:
%pwd

'/home/vaibhav/Chest_Cancer_Classification_Using_MLFlow-DVC/research'

In [4]:
os.chdir("../")

In [1]:
%pwd

'/home/vaibhav/End_To_End_Animal_Classification_Using_MLFlow&DVC/research'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    early_stopping_patience: int

In [7]:
# Update the configuration manager in src config
from Animal_Classification.constants import *
from Animal_Classification.utils.common import read_yaml,create_directories
import tensorflow as tf

2024-03-01 10:27:30.577892: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-01 10:27:30.580352: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 10:27:30.637991: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-01 10:27:30.639278: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 10:27:31.442326: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_roots])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "cat-vs-dog")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            early_stopping_patience=params.early_stopping_patience
        )

        return training_config

In [13]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from tensorflow.keras.callbacks import EarlyStopping

In [14]:


class Training:
    def __init__(self, config):
        self.config = config
        

    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # For validation data
        # validation_data_dir = Path(self.config.training_data)
        # self.valid_generator = valid_datagenerator.flow_from_directory(
        #     directory=str(validation_data_dir),
        #     shuffle=False,
        #     **dataflow_kwargs
        # )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        # For training data
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        training_data_dir = Path(self.config.training_data) 
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=str(training_data_dir),
            shuffle=True,
            **dataflow_kwargs
        )
        print(self.config.training_data)

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        early_stopping = EarlyStopping(
            monitor='val_loss',  # or another metric you want to monitor
            patience=self.config.early_stopping_patience,
            restore_best_weights=True
        )

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=[early_stopping]
        )

        self.save_model(path=self.config.trained_model_path, model=self.model)

       

In [15]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2024-03-01 10:28:37,710: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-01 10:28:37,713: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-01 10:28:37,714: INFO: common: created directory at: artifacts]
[2024-03-01 10:28:37,715: INFO: common: created directory at: artifacts/training]
Found 465 images belonging to 2 classes.
Found 465 images belonging to 2 classes.
artifacts/data_ingestion/kidney-ct-scan-image
Epoch 1/2


2024-03-01 10:28:38.595449: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


29/29 [==============================] - ETA: 0s - loss: 0.6956 - accuracy: 0.5880

2024-03-01 10:29:43.139323: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


29/29 [==============================] - 130s 4s/step - loss: 0.6956 - accuracy: 0.5880 - val_loss: 0.3523 - val_accuracy: 0.8815
Epoch 2/2
29/29 [==============================] - 124s 4s/step - loss: 0.6301 - accuracy: 0.6704 - val_loss: 0.3270 - val_accuracy: 0.8815
